In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
import seaborn as sns
# testing
from pandas.testing import assert_frame_equal
from tqdm import tqdm

import j_utils.munging as mg
from lendingclub.lc_utils import gen_datasets
from lendingclub import config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 60
pd.options.display.max_seq_items = None

# cleanup db script, until pandas adds an upsert

In [3]:
# %%writefile ../../lendingclub/investing/db_cleanup.py
# '''
# Temporary until pandas gets some kind of upsert functionality
# '''
# from sqlalchemy import create_engine
# from lendingclub import config
# import pandas as pd

# # read, sort, drop
# disk_engine = create_engine(f'sqlite:///{config.lc_api_db}')
# df = pd.read_sql('lc_api_loans', disk_engine)
# df = df.sort_values(['id', 'funded_amount'])
# df = df.drop_duplicates('id', keep='last')
# # df = df.drop(['index', 'level_0'], axis=1)
    

# # write out
# df.to_sql('lc_api_loans', disk_engine, if_exists='replace', index=False)

# one-time cleanup of db by adding useful date related columns

In [5]:
from sqlalchemy import create_engine
from lendingclub import config
from lendingclub.modeling import score_utils as scr_utils
from datetime import datetime #, timezone, timedelta
import pytz
import pandas as pd

now = datetime.now(pytz.FixedOffset(-480))

# read, sort, drop
disk_engine = create_engine(f'sqlite:///{config.lc_api_db}')
df = pd.read_sql('lc_api_loans', disk_engine)
# convert existing date cols
to_datify = [col for col in df.columns if '_d' in col and df[col].dtype == 'object']
for col in to_datify:
    df[col] = pd.to_datetime(df[col])
# add date cols: date, year, month, week of year, day, hour
df['last_seen_list_d'] = now
df['list_d_year'] = df['list_d'].dt.year
df['list_d_month'] = df['list_d'].dt.month
df['list_d_day'] = df['list_d'].dt.day
df['list_d_week'] = df['list_d'].dt.week
df['list_d_hour'] = df['list_d'].dt.hour
df['last_seen_list_d_year'] = df['last_seen_list_d'].dt.year
df['last_seen_list_d_month'] = df['last_seen_list_d'].dt.month
df['last_seen_list_d_day'] = df['last_seen_list_d'].dt.day
df['last_seen_list_d_week'] = df['last_seen_list_d'].dt.week
df['last_seen_list_d_hour'] = df['last_seen_list_d'].dt.hour

# write out
df.to_sql('lc_api_loans', disk_engine, if_exists='replace', index=False)

In [7]:
to_datify + ['last_seen_list_d']

['accept_d',
 'exp_d',
 'list_d',
 'credit_pull_d',
 'review_status_d',
 'ils_exp_d',
 'last_seen_list_d']

In [4]:
df

,id,member_id,loan_amount,funded_amount,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,is_inc_v,accept_d,exp_d,list_d,credit_pull_d,review_status_d,review_status,desc,purpose,addr_zip,addr_state,investor_count,ils_exp_d,initial_list_status,emp_title,acc_now_delinq,acc_open_past_24_mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2_yrs,delinq_amnt,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6_mths,mths_since_last_delinq,mths_since_last_record,mths_since_recent_inq,mths_since_recent_revol_delinq,mths_since_recent_bc,mort_acc,open_acc,pub_rec,total_bal_ex_mort,revol_bal,revol_util,total_bc_limit,total_acc,total_il_high_credit_limit,num_rev_accts,mths_since_recent_bc_dlq,pub_rec_bankruptcies,num_accts_ever_12_0_ppd,chargeoff_within_12_mths,collections_12_mths_ex_med,tax_liens,mths_since_last_major_derog,num_sats,num_tl_op_past_12m,mo_sin_rcnt_tl,tot_hi_cred_lim,tot_cur_bal,avg_cur_bal,num_bc_tl,num_actv_bc_tl,num_bc_sats,pct_tl_nvr_dlq,num_tl_90g_dpd_24m,num_tl_30dpd,num_tl_12_0dpd_2m,num_il_tl,mo_sin_old_il_acct,num_actv_rev_tl,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,total_rev_hi_lim,num_rev_tl_bal_gt_0,num_op_rev_tl,tot_coll_amt,application_type,annual_inc_joint,dti_joint,is_inc_v_joint,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,i_l_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,mtg_payment,housing_payment,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6_mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,disbursement_method,open_act_il,sec_app_open_act_il,catboost_regr,catboost_clf,catboost_regr_scl,catboost_comb_20,last_seen_list_d,list_d_year,list_d_month,list_d_day,list_d_week,list_d_hour,last_seen_list_d_year,last_seen_list_d_month,last_seen_list_d_day,last_seen_list_d_week,last_seen_list_d_hour
0,162593951,204027031,8800.0,7225.0,36,18.620001,12.86,1.52,320.89,D,D1,2 years,MORTGAGE,52000.0,SOURCE_VERIFIED,2019-12-16 06:49:27-08:00,2020-01-28 14:00:00-08:00,2019-12-29 14:00:00-08:00,2019-11-29 01:19:17-08:00,2019-12-29 09:55:18-08:00,APPROVED,None,debt_consolidation,034xx,NH,None,2019-12-29 14:00:00-08:00,F,Maintenance Technician,0.0,7,6174.0,25.000000,32.900002,37.230000,0,0.0,1997-11-28,665.0,669.0,2,38.0,NaN,2.0,38.0,3.0,1.0,12.0,0.0,86677.0,7836.0,45.000000,9200.0,14.0,83087.0,11.0,NaN,0.0,0,0.0,0.0,0.0,NaN,12.0,6.0,3.0,352184.0,334410.0,30401.0,5.0,4.0,5.0,92.0,0.0,0.0,0,2.0,9.0,8.0,264.0,3.0,17400.0,8.0,9.0,0.0,JOINT,131000.0,29.77,SOURCE_VERIFIED,3.0,2.0,2.0,5.0,78841.0,NaN,4.0,4.0,1834.0,45.000000,1.0,2.0,8.0,2070.89,NaN,655.0,659.0,1998-08-28,1.0,1.0,26.0,61.400002,24.0,0.0,0.0,NaN,42712.0,CASH,2.0,6.0,-0.205796,0.622922,0.617865,0.618877,2020-01-01 01:53:04.520148-08:00,2019,12,29,52,14,2020,1,1,1,1
1,162641707,204068568,32000.0,8600.0,36,8.810000,2.63,0.88,1014.77,A,A5,3 years,MORTGAGE,200000.0,NOT_VERIFIED,2019-12-29 09:52:53-08:00,2020-01-28 14:00:00-08:00,2019-12-29 14:00:00-08:00,2019-11-30 09:52:49-08:00,2019-12-29 09:54:19-08:00,APPROVED,None,home_improvement,124xx,NY,None,2019-12-29 14:00:00-08:00,F,Nurse Practitioner,0.0,8,4957.0,0.000000,30.200001,14.810000,2,0.0,2004-11-29,695.0,699.0,0,8.0,NaN,15.0,8.0,36.0,2.0,17.0,0.0,118347.0,12985.0,40.099998,7100.0,28.0,130843.0,12.0,NaN,0.0,1,0.0,0.0,0.0,67.0,17.0,2.0,2.0,414783.0,356179.0,20952.0,2.0,2.0,2.0,78.0,0.0,0.0,0,14.0,154.0,9.0,170.0,2.0,32400.0,9.0,11.0,0.0,JOINT,296000.0,13.04,NOT_VERIFIED,1.0,0.0,3.0,13.0,105362.0,76.5,2.0,5.0,1477.0,65.699997,1.0,4.0,0.0,1936.33,NaN,640.0,644.0,2003-12-29,0.0,2.0,20.0,44.099998,19.0,0.0,0.0,NaN,44727.0,CASH,5.0,5.0,-0.065238,0.868637,0.710957,0.742493,2020-01-01 01:53:04.520148-08:00,2019,12,29,52,14,2020,1,1,1,1
2,162647001,204073762,16000.0,5250.0,36,20.549999,12.86,1.52,599.12,D,D

In [42]:
df.select_dtypes(exclude=['datetime', 'float', 'int', 'object'])

,accept_d,exp_d,list_d,credit_pull_d,review_status_d,ils_exp_d,last_seen_list_d
0,2019-12-16 06:49:27-08:00,2020-01-28 14:00:00-08:00,2019-12-29 14:00:00-08:00,2019-11-29 01:19:17-08:00,2019-12-29 09:55:18-08:00,2019-12-29 14:00:00-08:00,2020-01-01 00:10:34.138546-08:00
1,2019-12-29 09:52:53-08:00,2020-01-28 14:00:00-08:00,2019-12-29 14:00:00-08:00,2019-11-30 09:52:49-08:00,2019-12-29 09:54:19-08:00,2019-12-29 14:00:00-08:00,2020-01-01 00:10:34.138546-08:00
2,2019-12-06 09:01:54-08:00,2020-01-20 09:54:49-08:00,2019-12-30 18:00:00-08:00,2019-11-30 13:55:14-08:00,2019-12-30 13:56:35-08:00,2019-12-30 18:00:00-08:00,2020-01-01 00:10:34.138546-08:00
3,2019-12-02 18:30:02-08:00,2020-01-16 18:31:30-08:00,2019-12-30 10:00:00-08:00,2019-12-02 18:28:47-08:00,2019-12-30 05:59:17-08:00,2019-12-30 10:00:00-08:00,2020-01-01 00:10:34.138546-08:00
4,2019-12-24 20:12:47-08:00,2020-01-26 18:00:00-08:00,2019-12-27 18:00:00-08:00,2019-12-03 06:01:20-08:00,2019-12-27 14:05:04-08:00,2019-12-27 18:00:00-08:00,2020-01-01 00:10:34.138546-08:00
...,...,...,...,...,...,...,...
113,2019-12-31 09:55:04-08:00,2020-01-30 14:00:00-08:00,2019-12-31 14:00:00-08:00,2019-12-31 09:53:45-08:00,2019-12-31 09:55:43-08:00,2019-12-31 14:00:00-08:00,2020-01-01 00:10:34.138546-08:00
114,2019-12-31 11:03:53-08:00,2020-01-30 18:00:00-08:00,2019-12-31 18:00:00-08:00,2019-12-31 10:59:56-08:00,2019-12-31 13:53:01-08:00,2019-12-31 18:00:00-08:00,2020-01-01 00:10:34.138546-08:00
115,2019-12-31 13:51:12-08:00,2020-01-30 18:00:00-08:00,2019-12-31 18:00:00-08:00,2019-12-31 13:39:12-08:00,2019-12-31 13:58:11-08:00,2019-12-31 18:00:00-08:00,2020-01-01 00:10:34.138546-08:00
116,2019-12-31 13:43:44-08:00,2020-01-30 18:00:00-08:00,2019-12-31 18:00:00-08:00,2019-12-31 13:37:47-08:00,2019-12-31 14:03:20-08:00,2019-12-31 18:00:00-08:00,2020-01-01 00:10:34.138546-08:00


In [ ]:
df.select_dtypes('datetime')

# Testing scripts below

In [31]:
df.query('catboost_comb_20 >= @scr_utils.min_comb_score')

,id,member_id,loan_amount,funded_amount,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,...,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,disbursement_method,open_act_il,sec_app_open_act_il,catboost_regr,catboost_clf,catboost_regr_scl,catboost_comb_20
70,164037904,206853699,14000.0,0.0,60,8.19,2.42,0.62,285.15,A,...,NaN,NaN,NaN,DIRECT_PAY,4.0,NaN,-0.010010,0.958588,0.747534,0.789745
88,164120976,206948945,4200.0,0.0,36,7.56,2.63,0.88,130.77,A,...,NaN,NaN,NaN,DIRECT_PAY,2.0,NaN,-0.008679,0.957437,0.748415,0.790220
102,164174325,207021632,9075.0,0.0,36,6.46,2.63,0.88,277.98,A,...,NaN,NaN,NaN,CASH,13.0,NaN,-0.018170,0.973354,0.742130,0.788375


In [84]:
mpath = config.modeling_dir

In [85]:
ls {mpath}

A_model_dataproc.pkl         C_model_dataproc.pkl  F_model_dataproc.pkl
A_model.pkl                  C_model.pkl           F_model.pkl
baseline_model_dataproc.pkl  D_model_dataproc.pkl  G_model_dataproc.pkl
baseline_model.pkl           D_model.pkl           G_model.pkl
B_model_dataproc.pkl         E_model_dataproc.pkl
B_model.pkl                  E_model.pkl


In [132]:
m = models.Model('B')
m.m

42

In [4]:
all_scores = pd.read_feather(os.path.join(config.data_dir, 'all_eval_loan_info_scored.fth'))
all_scores.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,catboost_regr_score,catboost_clf_score
0,2010-12-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.123967,0,0,paid,166683,A,36,0.041716,0.026271,0.011143,-0.003676,-0.018194,-0.032417,-0.046353,-0.060008,-0.073389,-0.086502,-0.099354,-0.111950,-0.124297,-0.136400,-0.148265,-0.159897,-0.171302,-0.182486,-0.193452,-0.204207,-0.214754,-0.225099,-0.235246,-0.245199,-0.254964,-0.264544,-0.273943,-0.283166,-0.292216,-0.301097,-0.309814,-0.039020,0.424992
1,2010-02-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.141801,0,0,paid,174377,B,36,0.065560,0.051043,0.036788,0.022790,0.009042,-0.004460,-0.017721,-0.030746,-0.043539,-0.056106,-0.068451,-0.080578,-0.092492,-0.104196,-0.115696,-0.126996,-0.138098,-0.149008,-0.159728,-0.170264,-0.180617,-0.190793,-0.200794,-0.210624,-0.220286,-0.229784,-0.239121,-0.248299,-0.257323,-0.266194,-0.274917,-0.058365,0.574708
2,2010-12-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.237031,0,0,paid,178446,E,36,0.146518,0.129520,0.112871,0.096562,0.080585,0.064931,0.049594,0.034566,0.019840,0.005408,-0.008735,-0.022598,-0.036187,-0.049507,-0.062565,-0.075367,-0.087919,-0.100227,-0.112296,-0.124132,-0.135740,-0.147125,-0.158293,-0.169247,-0.179994,-0.190538,-0.200882,-0.211032,-0.220993,-0.230767,-0.240360,-0.076193,0.782984
3,2008-12-01,2008-01-01,0.191822,1.0,1.0,1.0,14147.749023,0.246457,1,1,charged_off,209667,E,36,-0.758478,-0.759404,-0.760324,-0.761238,-0.762146,-0.763049,-0.763946,-0.764837,-0.765723,-0.766603,-0.767477,-0.768346,-0.769210,-0.770068,-0.770921,-0.771768,-0.772610,-0.773447,-0.774279,-0.775106,-0.775928,-0.776744,-0.777556,-0.778362,-0.779164,-0.779961,-0.780753,-0.781540,-0.782322,-0.783100,-0.783873,-0.046314,0.673052
4,2011-02-01,2008-01-01,1.000000,1.0,1.0,1.0,0.000000,1.262633,0,0,paid,223015,F,36,0.170240,0.152890,0.135895,0.119247,0.102938,0.086960,0.071305,0.055965,0.040933,0.026202,0.011765,-0.002386,-0.016256,-0.029852,-0.043181,-0.056249,-0.069061,-0.081625,-0.093944,-0.106025,-0.117874,-0.129495,-0.140894,-0.152076,-0.163045,-0.173807,-0.184366,-0.194727,-0.204893,-0.214871,-0.224662,-0.074388,0.470468
